## Supply Chain simulation demo

This notebook uses the supplychain-simulation package contained in this repository.
To install the simulator into the current kernel, run the next cell.

Jupyter will not pick up any changes made to the package automatically.  
To load your changes into the notebook:
1) run the next cell
2) restart the kernel (so the newly installed package is re-loaded)

In [ ]:
! pip install ../

In [ ]:
from supplychain_simulation import SupplyChain, Node, Sales, LeadTime, Edge, Simulator
from supplychain_simulation.node import Orders, Stock
from supplychain_simulation.pipeline import Pipeline, Receipt
from supplychain_simulation.strategy import RSQ, Fractional

### Define the supply chain.

The only manadtory field for a Node is it's `id`.

#### Sales
To simulate demand over time for a node, you can provide the `sales` field with an object that defines a `pop_sales(period: int)` method returning the sales for that period.  
A convenience class `Sales` is provided to cover most non-dynamic use cases.
It accepts a dict where the key is the period and the value is the sales for that period.

#### Lead-time
To simulate changes in lead-time over time for a node, you can provide the `lead_time` field with an object that defines a `get_lead_time(period: int)` method returning the lead-time for that period.  
A conveniance class `LeadTime` is provided to cover most non-dynamic use cases.
It accepts a dict where the key is the period and the value is the lead-time for that period.
in addition it also accepts a `default` that will be used for any undefined period.

#### Edges
The edges can be defined in two ways, through `Node.predecessors` and/or through `SupplyChain.edges`.
If both are present they will both be applied to the supplychain with `Node.predecessors` taking precedence.

If an `Edge` is defined while the corresponding `Node`'s don't exist, an error will be raised.


In [ ]:
supply_chain = SupplyChain(
        nodes=[
            Node(
                "A",
                data=dict(
                    order_quantity=30,
                    reorder_level=25,
                    review_time=1,
                    safety_stock=1,
                ),
                llc=0,
                sales=Sales({1: [10], 2: [10], 3: [10]}),
                lead_time=LeadTime(default=1),
                stock=Stock({"A": 7, "C": 5}),
                pipeline=Pipeline([Receipt(sku_code="D", eta=0, quantity=35)]),
            ),
            Node(
                "B",
                data=dict(
                    order_quantity=25,
                    reorder_level=40,
                    review_time=1,
                ),
                llc=0,
                sales=Sales({1: [15], 2: [15], 3: [15]}),
                lead_time=LeadTime(default=2),
                pipeline=Pipeline([Receipt(sku_code="D", eta=1, quantity=75)]),
                backorders=5,
            ),
            Node(
                "C",
                data=dict(
                    order_quantity=150,
                    reorder_level=20,
                    review_time=1,
                ),
                llc=1,
                lead_time=LeadTime({1: 3, 2: 7}, default=3),
                stock=Stock({"C": 200}),
            ),
            Node(
                "D",
                data=dict(
                    order_quantity=200,
                    reorder_level=20,
                    review_time=2,
                ),
                llc=1,
                lead_time=LeadTime(default=4),
                orders=Orders({"B": 15}),
                stock=Stock({"D": 40}),
                pipeline=Pipeline([Receipt(sku_code="D", eta=2, quantity=200)]),
            ),
        ],
        edges=[
            Edge(source="C", destination="A", number=2),
            Edge(source="D", destination="A", number=1),
            Edge(source="D", destination="B", number=3),
        ],
    )

Keep in mind the simulation alters the supply-chain in-place so running the simulator twice on the same object might not give the same results.

In [ ]:
sim = Simulator(
    supply_chain,
    control_strategy=RSQ(supply_chain),
    release_strategy=Fractional(),
)

In [ ]:
sim.run(10)